In [1]:
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.tools import QueryEngineTool

# settings
Settings.llm = OpenAI(model="gpt-3.5-turbo",temperature=0)

In [2]:
# function tools
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

In [7]:
import os
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.embeddings.openai import OpenAIEmbedding

def load_or_create_index(directory_path, persist_dir):
        if os.path.exists(persist_dir):
            print("Loading existing index...")
            storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print("Creating new index...")
            documents = SimpleDirectoryReader(directory_path, recursive=True).load_data()
            index = VectorStoreIndex.from_documents(
                documents=documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
            )
            index.storage_context.persist(persist_dir=persist_dir)
        return index

index = load_or_create_index(
            "data/rag/",
            "storage"
        )
query_engine = index.as_query_engine()

budget_tool = QueryEngineTool.from_defaults(
    query_engine, 
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget."
)

Creating new index...


In [11]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, budget_tool], verbose=True)

response = query_engine.query("How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?")
print(response)

$20 billion was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget.


In [13]:
from llama_parse import LlamaParse

import nest_asyncio
nest_asyncio.apply()

documents2 = LlamaParse(result_type="markdown").load_data(
    "data/rag/2023_canadian_budget.pdf"
)
index2 = VectorStoreIndex.from_documents(documents2)
query_engine2 = index2.as_query_engine()

response2 = query_engine2.query(
    "How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?"
)
print(response2)

Started parsing the file under job_id 6b1e93b9-5ace-480b-a1eb-61a068abcd79
$20 billion was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget.
